# Data Prepare and Exploring
## Download Data
Before running this, download 

http://s3.kiva.org/snapshots/kiva_ds_csv.zip

to Dwonloads subdir of your home directory. Then extract the zip file in the same diretory.

This script should work for both Windows and Mac OS, as long as you follow the above instructions.
# Create sqlite
create db/kiva_data.sqlite

In [33]:
from datetime import datetime, timezone
import math

import numpy as np
import pandas as pd

from os import path
from pathlib import Path

import sqlalchemy as sa
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.orm import sessionmaker

import pymysql
pymysql.install_as_MySQLdb()

from config import MYSQL_URL

DB = 'mysql'

if DB != 'mysql':
    engine = create_engine("sqlite:///db/kiva.sqlite")
else:
    engine = create_engine(MYSQL_URL, encoding='utf-8')

conn = engine.connect()
metadata = MetaData()

HOME_DIR = str(Path.home())
KIVA_PATH = path.join(HOME_DIR, 'Downloads', 'kiva_ds_csv')

## Load lenders into DB

In [34]:
DB

'mysql'

In [35]:
lender_df = pd.read_csv(path.join(KIVA_PATH, 'lenders.csv')) #.head(1000)
lender_df.columns = map(str.lower, lender_df.columns)

# member_since is UTC time, change to standard format
lender_df.loc[:, 'member_since'] = lender_df['member_since'].map(lambda x: datetime.utcfromtimestamp(x))

# Assign numeric ID to lender to reduce space needed for loan_lender table
lender_df = lender_df.assign(lender_id=np.arange(len(lender_df)))
lender_df = lender_df[['lender_id', 'permanent_name', 'display_name', 'city', 'state', 'country_code', 'member_since']]

In [36]:
lender_df.columns

Index(['lender_id', 'permanent_name', 'display_name', 'city', 'state',
       'country_code', 'member_since'],
      dtype='object')

In [3]:
if DB == 'mysql':
    conn.execute('truncate lender')
else:
    conn.execute('delete from lender')
    
lender_df.to_sql('lender', engine, if_exists='append', index=False)

KeyboardInterrupt: 

## Load loans into DB

In [37]:
loan_df = pd.read_csv(path.join(KIVA_PATH, 'loans.csv')) #.head(1000)
loan_df.columns = map(str.lower, loan_df.columns)

In [ ]:
loan_df.columns

In [39]:
## Convert gender to be F(femal), M(male) or U (unknown)
def map_gender(x):
    if (isinstance(x, str)): 
        return 'F' if x.startswith('female') else 'M'
    else:
        return 'U'
    
loan_df.loc[:, 'gender'] = loan_df['borrower_genders'].map(map_gender)    

# Write only needed column to DB as number of rows is large
# loan_df = loan_df[['loan_id', 'loan_name', 'loan_amount', 'gender', 'country_code', 'posted_time']]

# Trim datetime to avoid issue with Mysql
loan_df['posted_time'] = loan_df['posted_time'].map(lambda x: x[:len('2014-01-15 02:23:45')])    
#conn.execute('truncate loan') 
loan_df.to_sql('loan', engine,  if_exists='replace', index=False)

In [40]:
loan_df.columns

Index(['loan_id', 'loan_name', 'original_language', 'description',
       'description_translated', 'funded_amount', 'loan_amount', 'status',
       'image_id', 'video_id', 'activity_name', 'sector_name', 'loan_use',
       'country_code', 'country_name', 'town_name', 'currency_policy',
       'currency_exchange_coverage_rate', 'currency', 'partner_id',
       'posted_time', 'planned_expiration_time', 'disburse_time',
       'raised_time', 'lender_term', 'num_lenders_total',
       'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_names',
       'borrower_genders', 'borrower_pictured', 'repayment_interval',
       'distribution_model', 'gender'],
      dtype='object')

In [41]:
sqls = ["create index loan_id_idx on loan(loan_id)",
        "create index posted_time_idx on loan(posted_time)"
       ]
for sql in sqls:
    print(sql)
    engine.execute(sql)

create index loan_id_idx on loan(loan_id)
create index posted_time_idx on loan(posted_time)


InternalError: (pymysql.err.InternalError) (1170, "BLOB/TEXT column 'posted_time' used in key specification without a key length") [SQL: 'create index posted_time_idx on loan(posted_time)'] (Background on this error at: http://sqlalche.me/e/2j85)

## Load loan lender links to DB

In [14]:
loan_lender_df = pd.read_csv(path.join(KIVA_PATH, 'loans_lenders.csv'))#.head(1000)
loan_lender_df.columns = map(str.lower, loan_lender_df.columns)

# break lenders into lender_list
loan_lender_df.loc[:, 'lender_list'] = loan_lender_df['lenders'].map(
    lambda x: x.split(', ')
)

In [15]:
# Build a quick lookup from lender name to ID
lender_id_lookup = {
    x['permanent_name']: x['lender_id']
    for i, x in lender_df.iterrows()
}

In [16]:
# Convert lender names to lender ID's
loan_lender_df.loc[:, 'lender_id_list'] = loan_lender_df['lender_list'].map(
    lambda lenders: [lender_id_lookup.get(lender, 0) for lender in lenders]
)

In [17]:
# Create a new data frame holding loan_id and lender_id
df = loan_lender_df[['loan_id', 'lender_id_list']]

In [18]:
# Create loan_id_list same len as lender_id_list
df.loc[:, 'loan_id_list'] = df.apply(lambda x: [x[0]] * len(x[1]), axis=1, reduce=True)

C:\Users\feng443\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\feng443\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [19]:
# Ref: https://stackoverflow.com/questions/35004945/python-pandas-reduce-function-for-series
from itertools import chain # Based on stackoverflow, chain is 68x faster than reduce

In [20]:
combo_df = pd.DataFrame( {
    'loan_id': list(chain(*df.loc[:, 'loan_id_list'])),
    'lender_id': list(chain(*df.loc[:, 'lender_id_list'])),
})

In [21]:
# Write to database table
# Disable constraint before insert to speed up performance
#combo_df.to_sql('loan_lender', engine, if_exists='append', index=False)
#insert_df(combo_df, 'loan_lender', engine, if_exists='append', index=False)

combo_df = combo_df.drop_duplicates()

def bulk_insert_df(df, table_name):
    Session = sessionmaker(bind=engine)
    session = Session()
    table = Table(table_name, metadata, autoload=True, autoload_with=engine)
    
    parts = 100
    chunksize = math.floor(df.shape[0] / parts)
    chunks = [(chunksize * i, (chunksize * i) + chunksize) for i in range(parts)]
    if chunksize * parts < df.shape[0]:
        chunks.append((chunksize * parts, df.shape[0]))
  
    
    for chunk in chunks:
        i, j = chunk
        print(f'{i}: {j}')
        listToWrite = df.iloc[i:j, :].to_dict(orient='records')
        # to_sql has trouble with numpy::int64 data type
        listToWrite = [ {y: int(z) for y,z in x.items()} for x in listToWrite]
        conn.execute(table.insert(), listToWrite)
        session.commit()
    
    session.close()
    
if DB == 'mysql':
    conn.execute('truncate loan_lender')
else:
    conn.execute('delete from loan_lender')
bulk_insert_df(combo_df.astype('int'), 'loan_lender')
print('Done')

0: 274590
274590: 549180
549180: 823770
823770: 1098360
1098360: 1372950
1372950: 1647540
1647540: 1922130
1922130: 2196720
2196720: 2471310
2471310: 2745900
2745900: 3020490
3020490: 3295080
3295080: 3569670
3569670: 3844260
3844260: 4118850
4118850: 4393440
4393440: 4668030
4668030: 4942620
4942620: 5217210
5217210: 5491800
5491800: 5766390
5766390: 6040980
6040980: 6315570
6315570: 6590160
6590160: 6864750
6864750: 7139340
7139340: 7413930
7413930: 7688520
7688520: 7963110
7963110: 8237700
8237700: 8512290
8512290: 8786880
8786880: 9061470
9061470: 9336060
9336060: 9610650
9610650: 9885240
9885240: 10159830
10159830: 10434420
10434420: 10709010
10709010: 10983600
10983600: 11258190
11258190: 11532780
11532780: 11807370
11807370: 12081960
12081960: 12356550
12356550: 12631140
12631140: 12905730
12905730: 13180320
13180320: 13454910
13454910: 13729500
13729500: 14004090
14004090: 14278680
14278680: 14553270
14553270: 14827860
14827860: 15102450
15102450: 15377040
15377040: 15651630
15

In [22]:
conn.close()